<a href="https://colab.research.google.com/github/Lishat/Human-Action-Classification-using-Tensorflow/blob/master/Convolution_Neural_Networks_in_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import cv2
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
m=sum([len(files) for r, d, files in os.walk("/content/drive/My Drive/Assignment-1_Datasets /A1-Q2_Dataset/HumanActionClassification/")])
X = np.zeros((m, 224, 224, 3))
Y = np.zeros((m, 7))
count = 0
for i in range(1, 8):
    for file in os.listdir("/content/drive/My Drive/Assignment-1_Datasets /A1-Q2_Dataset/HumanActionClassification/"+str(i)+"/"):
        X[count] = cv2.resize(cv2.imread("/content/drive/My Drive/Assignment-1_Datasets /A1-Q2_Dataset/HumanActionClassification/"+str(i)+"/"+file), dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
        Y[count][i-1] = 1
        count += 1
        print(count)
X = X[:count]
Y = Y[:count]
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size = 0.25, random_state = 0)

In [0]:
def Convolution_Network(x_train, x_test, y_train, y_test):
  x = tf.placeholder(tf.float32, [None, 224, 224, 3])
  y = tf.placeholder(tf.float32, [None, 7])
  W1 = tf.Variable(tf.truncated_normal([5, 5, 3, 16], stddev = 0.05), name='W1')
  b1 = tf.Variable(tf.truncated_normal([16]), name='b1')
  z1 = tf.nn.relu(tf.nn.conv2d(x , W1, [1, 1, 1, 1], padding='SAME') + b1)
  a1 = tf.nn.max_pool(z1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="SAME")
  W2 = tf.Variable(tf.truncated_normal([5, 5, 16, 32], stddev = 0.05), name='W2')
  b2 = tf.Variable(tf.truncated_normal([32]), name='b2')
  z2 = tf.nn.relu(tf.nn.conv2d(a1 , W2, [1, 1, 1, 1], padding='SAME') + b2)
  a2 = tf.nn.max_pool(z2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="SAME")
  W3 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev = 0.05), name='W3')
  b3 = tf.Variable(tf.truncated_normal([64]), name='b3')
  z3 = tf.nn.relu(tf.nn.conv2d(a2 , W3, [1, 1, 1, 1], padding='SAME') + b3)
  a3 = tf.nn.max_pool(z3, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="SAME")
  W4 = tf.Variable(tf.truncated_normal([5, 5, 64, 128], stddev = 0.05), name='W4')
  b4 = tf.Variable(tf.truncated_normal([128]), name='b4')
  z4 = tf.nn.relu(tf.nn.conv2d(a3 , W4, [1, 1, 1, 1], padding='SAME') + b4)
  a4 = tf.nn.max_pool(z4, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="SAME")
  W5 = tf.Variable(tf.truncated_normal([5, 5, 128, 256], stddev = 0.05), name='W5')
  b5 = tf.Variable(tf.truncated_normal([256]), name='b5')
  z5 = tf.nn.relu(tf.nn.conv2d(a4 , W5, [1, 1, 1, 1], padding='SAME') + b5)
  a5 = tf.nn.max_pool(z5, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="SAME")
  flatten = tf.reshape(a5, [-1, 7*7*256])
  W6 = tf.Variable(tf.truncated_normal([7*7*256, 256], stddev = 0.05), name='W6')
  b6 = tf.Variable(tf.truncated_normal([256]), name='b6')
  a6 = tf.nn.relu(tf.matmul(flatten, W6) + b6)
  W7 = tf.Variable(tf.truncated_normal([256, 7], stddev = 0.05), name='W7')
  b7 = tf.Variable(tf.truncated_normal([7]), name='b7')
  z7 = tf.matmul(a6, W7) + b7
  a7 = tf.nn.softmax(z7)
  cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=z7, labels = y))
  optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cross_entropy_loss)
  accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y, 1), tf.argmax(a7, 1)), tf.float32))
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batch_size = 64
    total_batch = x_train.shape[0]//batch_size
    for epoch in range(1000):
        avg_cost = 0
        for i in range(total_batch):
            _, c = sess.run([optimizer, cross_entropy_loss], feed_dict={x:xTrain[i*batch_size:(i+1)*batch_size], y:yTrain[i*batch_size:(i+1)*batch_size]})
            avg_cost += c/total_batch
        train_accuracy = sess.run(accuracy, feed_dict={x:xTrain, y:yTrain})
        test_accuracy = sess.run(accuracy, feed_dict={x:xTest, y:yTest})
        print("Epoch:", (epoch+1))
        print("Cost:", avg_cost)
        print("Train accuracy:", train_accuracy)
        print("Test accuracy:", test_accuracy)

In [0]:
Convolution_Network(xTrain, xTest, yTrain, yTest)